In [2]:
# Read Data
def read_data():
    f = open("data/bezdekIris.data", "r")
    fData = f.readlines()
    lData = []

    for sIter in fData:
        if (sIter[-1] == "\n"):
            sIter = sIter[:-1]
        if (sIter == ""):
            continue
        lData.append(sIter.split(","))
    return lData

5


In [3]:
# Create HashMaps
def create_dictionary(lData):
    lDicts = [dict() for x in range(len(lData[0]))] # array/list of dictionaries (Hashmaps)
    for i in range(0, len(lData[0])):
        dFeatures = {} # Empty hashmaps
        for j in range(0, len(lData)):
            if lData[j][i] in dFeatures.keys():
                dFeatures[lData[j][i]] = dFeatures[lData[j][i]] + 1
            else:
                dFeatures[lData[j][i]] = 1
        lDicts[i] = dFeatures
    return lDicts

In [4]:
from copy import deepcopy
nIcebergCondn = 10
lData1 = deepcopy(lData)
for i in range(0, len(lData[0])):
    dInterest = lDicts[i]
    for j in range(0, len(lData)):
        if (dInterest[lData[j][i]] < nIcebergCondn):
            lData1[j][i] = '*';
print(lData1)

[['*', '*', '1.4', '0.2', 'Iris-setosa'], ['*', '3.0', '1.4', '0.2', 'Iris-setosa'], ['*', '3.2', '*', '0.2', 'Iris-setosa'], ['*', '3.1', '1.5', '0.2', 'Iris-setosa'], ['5.0', '*', '1.4', '0.2', 'Iris-setosa'], ['*', '*', '*', '*', 'Iris-setosa'], ['*', '3.4', '1.4', '*', 'Iris-setosa'], ['5.0', '3.4', '1.5', '0.2', 'Iris-setosa'], ['*', '2.9', '1.4', '0.2', 'Iris-setosa'], ['*', '3.1', '1.5', '*', 'Iris-setosa'], ['*', '*', '1.5', '0.2', 'Iris-setosa'], ['*', '3.4', '*', '0.2', 'Iris-setosa'], ['*', '3.0', '1.4', '*', 'Iris-setosa'], ['*', '3.0', '*', '*', 'Iris-setosa'], ['*', '*', '*', '0.2', 'Iris-setosa'], ['*', '*', '1.5', '*', 'Iris-setosa'], ['*', '*', '*', '*', 'Iris-setosa'], ['*', '*', '1.4', '*', 'Iris-setosa'], ['*', '*', '*', '*', 'Iris-setosa'], ['*', '*', '1.5', '*', 'Iris-setosa'], ['*', '3.4', '*', '0.2', 'Iris-setosa'], ['*', '*', '1.5', '*', 'Iris-setosa'], ['*', '*', '*', '0.2', 'Iris-setosa'], ['*', '*', '*', '*', 'Iris-setosa'], ['*', '3.4', '*', '0.2', 'Iris-se

In [5]:
lData2 = deepcopy(lData1)
for i in range(0, len(lData2)):
    lData2[i] = tuple(lData2[i])
lData2

[('*', '*', '1.4', '0.2', 'Iris-setosa'),
 ('*', '3.0', '1.4', '0.2', 'Iris-setosa'),
 ('*', '3.2', '*', '0.2', 'Iris-setosa'),
 ('*', '3.1', '1.5', '0.2', 'Iris-setosa'),
 ('5.0', '*', '1.4', '0.2', 'Iris-setosa'),
 ('*', '*', '*', '*', 'Iris-setosa'),
 ('*', '3.4', '1.4', '*', 'Iris-setosa'),
 ('5.0', '3.4', '1.5', '0.2', 'Iris-setosa'),
 ('*', '2.9', '1.4', '0.2', 'Iris-setosa'),
 ('*', '3.1', '1.5', '*', 'Iris-setosa'),
 ('*', '*', '1.5', '0.2', 'Iris-setosa'),
 ('*', '3.4', '*', '0.2', 'Iris-setosa'),
 ('*', '3.0', '1.4', '*', 'Iris-setosa'),
 ('*', '3.0', '*', '*', 'Iris-setosa'),
 ('*', '*', '*', '0.2', 'Iris-setosa'),
 ('*', '*', '1.5', '*', 'Iris-setosa'),
 ('*', '*', '*', '*', 'Iris-setosa'),
 ('*', '*', '1.4', '*', 'Iris-setosa'),
 ('*', '*', '*', '*', 'Iris-setosa'),
 ('*', '*', '1.5', '*', 'Iris-setosa'),
 ('*', '3.4', '*', '0.2', 'Iris-setosa'),
 ('*', '*', '1.5', '*', 'Iris-setosa'),
 ('*', '*', '*', '0.2', 'Iris-setosa'),
 ('*', '*', '*', '*', 'Iris-setosa'),
 ('*', '3.

In [6]:
dFeatures = {}

for i in range(0, len(lData2)):
    if lData2[i] in dFeatures.keys():
        dFeatures[lData2[i]] = dFeatures[lData2[i]] + 1
    else:
        dFeatures[lData2[i]] = 1
dFeatures

{('*', '*', '1.4', '0.2', 'Iris-setosa'): 2,
 ('*', '3.0', '1.4', '0.2', 'Iris-setosa'): 1,
 ('*', '3.2', '*', '0.2', 'Iris-setosa'): 3,
 ('*', '3.1', '1.5', '0.2', 'Iris-setosa'): 2,
 ('5.0', '*', '1.4', '0.2', 'Iris-setosa'): 2,
 ('*', '*', '*', '*', 'Iris-setosa'): 6,
 ('*', '3.4', '1.4', '*', 'Iris-setosa'): 1,
 ('5.0', '3.4', '1.5', '0.2', 'Iris-setosa'): 1,
 ('*', '2.9', '1.4', '0.2', 'Iris-setosa'): 1,
 ('*', '3.1', '1.5', '*', 'Iris-setosa'): 1,
 ('*', '*', '1.5', '0.2', 'Iris-setosa'): 3,
 ('*', '3.4', '*', '0.2', 'Iris-setosa'): 3,
 ('*', '3.0', '1.4', '*', 'Iris-setosa'): 2,
 ('*', '3.0', '*', '*', 'Iris-setosa'): 1,
 ('*', '*', '*', '0.2', 'Iris-setosa'): 4,
 ('*', '*', '1.5', '*', 'Iris-setosa'): 4,
 ('*', '*', '1.4', '*', 'Iris-setosa'): 2,
 ('5.0', '3.0', '*', '0.2', 'Iris-setosa'): 1,
 ('5.0', '3.4', '*', '*', 'Iris-setosa'): 1,
 ('*', '3.4', '1.4', '0.2', 'Iris-setosa'): 1,
 ('*', '3.1', '*', '0.2', 'Iris-setosa'): 1,
 ('*', '3.4', '1.5', '*', 'Iris-setosa'): 1,
 ('5.0

In [7]:
lKeys = sorted(dFeatures.keys())
lKeys

[('*', '*', '*', '*', 'Iris-setosa'),
 ('*', '*', '*', '*', 'Iris-versicolor'),
 ('*', '*', '*', '*', 'Iris-virginica'),
 ('*', '*', '*', '0.2', 'Iris-setosa'),
 ('*', '*', '*', '1.3', 'Iris-versicolor'),
 ('*', '*', '*', '1.5', 'Iris-versicolor'),
 ('*', '*', '*', '1.5', 'Iris-virginica'),
 ('*', '*', '*', '1.8', 'Iris-virginica'),
 ('*', '*', '1.4', '*', 'Iris-setosa'),
 ('*', '*', '1.4', '0.2', 'Iris-setosa'),
 ('*', '*', '1.5', '*', 'Iris-setosa'),
 ('*', '*', '1.5', '0.2', 'Iris-setosa'),
 ('*', '2.8', '*', '*', 'Iris-versicolor'),
 ('*', '2.8', '*', '*', 'Iris-virginica'),
 ('*', '2.8', '*', '1.3', 'Iris-versicolor'),
 ('*', '2.8', '*', '1.5', 'Iris-versicolor'),
 ('*', '2.8', '*', '1.5', 'Iris-virginica'),
 ('*', '2.8', '*', '1.8', 'Iris-virginica'),
 ('*', '2.9', '*', '*', 'Iris-versicolor'),
 ('*', '2.9', '*', '1.3', 'Iris-versicolor'),
 ('*', '2.9', '*', '1.5', 'Iris-versicolor'),
 ('*', '2.9', '*', '1.8', 'Iris-virginica'),
 ('*', '2.9', '1.4', '0.2', 'Iris-setosa'),
 ('*', 

In [8]:
#Tree class
class Tree(object):
    def __init__(node, label = "all", children = None, count = 0):
        node.name = label
        node.children = []
        node.count = count
        if (children != None):
            for i in (0, len(children)):
                node.add_node(children[i])
    
    def add_node(node, child):
        node.children.append(child)
    
    def find(node, label = str("")):
        for i in range(0, len(node.children)):
            if (node.children[i].name == label):
                return node.children[i]
        return None
    def update_count(node, count = 0):
        node.count += count
        

In [14]:
def create_star_tree(lKeys):
    star_tree = Tree()
    for i in range (0, len(lKeys)):
        head = star_tree
        add_children_to_tree(head, lKeys[i], 0)
    return star_tree

In [12]:
def add_children_to_tree(head, tChars, nIndex):
    tNew = head.find(tChars[nIndex])
    if (tNew != None):
        tNew.update_count(count = dFeatures[tChars])
    else:
        tNew = Tree(label = tChars[nIndex], count = dFeatures[tChars])
        head.add_node(tNew)
    nIndex += 1
    if (nIndex < len(tChars)):
        add_children_to_tree(tNew, tChars, nIndex)

In [35]:
star_tree = create_star_tree(lKeys)
print(star_tree.children[0].children[0].count)
fPrintTree(star_tree)

58
Iris-setosa 6
Iris-versicolor 14
Iris-virginica 14
* 34
Iris-setosa 4
0.2 4
Iris-versicolor 4
1.3 4
Iris-versicolor 2
Iris-virginica 1
1.5 3
Iris-virginica 2
1.8 2
* 47
Iris-setosa 2
* 2
Iris-setosa 2
0.2 2
1.4 4
Iris-setosa 4
* 4
Iris-setosa 3
0.2 3
1.5 7
* 58
Iris-versicolor 2
Iris-virginica 6
* 8
Iris-versicolor 3
1.3 3
Iris-versicolor 1
Iris-virginica 1
1.5 2
Iris-virginica 1
1.8 1
* 14
2.8 14
Iris-versicolor 1
* 1
Iris-versicolor 5
1.3 5
Iris-versicolor 1
1.5 1
Iris-virginica 2
1.8 2
* 9
Iris-setosa 1
0.2 1
1.4 1
2.9 10
Iris-setosa 1
Iris-versicolor 4
Iris-virginica 8
* 13
Iris-setosa 1
0.2 1
Iris-versicolor 1
1.3 1
Iris-versicolor 3
1.5 3
Iris-virginica 4
1.8 4
* 22
Iris-setosa 2
* 2
Iris-setosa 1
0.2 1
1.4 3
3.0 25
Iris-versicolor 1
Iris-virginica 3
* 4
Iris-setosa 1
0.2 1
Iris-versicolor 2
1.5 2
Iris-virginica 1
1.8 1
* 8
Iris-setosa 1
* 1
Iris-setosa 2
0.2 2
1.5 3
3.1 11
Iris-versicolor 1
Iris-virginica 4
* 5
Iris-setosa 3
0.2 3
Iris-versicolor 1
1.5 1
Iris-versicolor 1
Iri

In [34]:
def fPrintTree(tree):
    nIndex = 0
    if (len(tree.children) != 0):
        for child in tree.children:
            fPrintTree(tree.children[nIndex])
            nIndex+=1
    print(tree.name, tree.count)